In [0]:
!pip install -q tensorflow tensorflow-datasets matplotlib

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import scipy as scipy

print("tensorflow version: " + tf.__version__)

tf.compat.v1.enable_eager_execution()
np.set_printoptions(linewidth=300)

In [0]:
(x_train, y_train), (x_test, y_test)  = tf.keras.datasets.mnist.load_data()
#(x_train, y_train), (x_test, y_test)  = tf.keras.datasets.cifar10.load_data()
print (f'\nInitial dataset: train - {x_train.shape} : {y_train.shape}, test - {x_test.shape} : {y_test.shape}')

In [0]:
print(f'\nInput image for digit {y_train[0]} is {x_train[0].shape} shape:')
print (x_train[0])

In [0]:
normalizedX = (x_train.astype(np.float32) - 128) / 128
print (f'\nNormalized dataset: {normalizedX.shape}')
print(f'\nNormalized image for digit {y_train[0]} is {normalizedX[0].shape} shape:')
print (normalizedX[0].round (1))

In [0]:
reshapedNormalizedX = normalizedX.reshape (len(normalizedX),normalizedX.shape[1] * normalizedX.shape[2])
print (f'\nNormalized images as vectors dataset: {reshapedNormalizedX.shape}')

In [0]:
x = np.hstack((reshapedNormalizedX,np.ones((reshapedNormalizedX.shape[0],1))))
print (f'\nNormalized images with as vectors dataset with bias trick: {x.shape}')

In [0]:
classes = np.unique (y_train)
print (f'\nclasses: {classes}')

In [0]:
classesQuantity = len(classes)
print (f'\nClasses quantity: {classesQuantity}')

In [0]:
y = np.zeros((len(y_train), classesQuantity))
y [np.arange(len(y_train)), y_train] = 1
print (f'\nClasses reshaped: {y.shape}')
print (y)

In [0]:
w = np.zeros([x.shape[1], classesQuantity])
print (f'\nInitial weighs: {w.shape} are zeros')
print (w.round (2))

In [0]:
regularization = 0.1
learningRage = 0.01

def batchSoftmax (z):
    z -= np.max(z)
    sm = (np.exp(z).T / np.sum(np.exp(z), axis=1)).T
    return sm

def itemSoftmax (x):
    return np.exp(x) / np.sum(np.exp(x))

def getLoss (w, x, y):
    batchSize = x.shape[0]
    scores = np.dot(x,w)
    prob = batchSoftmax(scores)
    loss = (-1 / batchSize) * np.sum(y * np.log(prob)) + (regularization / 2) * np.sum (w * w)
    grad = (-1 / batchSize) * np.dot(x.T,(y - prob)) + regularization * w
    return loss,grad

for i in range(0,10):
    loss,grad = getLoss(w,x,y)
    w = w - (learningRage * grad)

    gt = y_train[i]
    prob = itemSoftmax(x[i] @ w)
    argmax = np.argmax (prob)

    print(f'Loss: {loss.round(2)}, Class: {gt},  Pred: {argmax}, Pred prob: {prob[argmax].round(2)}, Class probe: {prob[gt].round(2)}')
